## Preprocess

In [2]:
import mido
from mido import Message, MidiFile, MidiTrack,MetaMessage
import string
import numpy as np
import pandas as pd
from midi_array import *
import os
import csv

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [4]:
# label path     
label_path = r'D:\BrownUniversity\CS2470\final_proj\CS2470_final\data\label.csv'
# Define the folder path
folder_path = r'D:\BrownUniversity\CS2470\final_proj\CS2470_final\data\test'
# load data
music,tag = get_music_data(folder_path, label_path)

In [4]:
print(f"music_shape:{music.shape}")

music_shape:(31, 10000, 88)


In [9]:
def one_hot_encode(labels, num_classes):
    one_hot_labels = np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        one_hot_labels[i, label - 1] = 1
    return one_hot_labels

## VAE 

In [23]:
class VAE(nn.Module):
    def __init__(self, input_size, hidden_dim, latent_size, label_size):
        super(VAE, self).__init__()
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.latent_size = latent_size
        self.label_size = label_size

        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        self.mu_layer = nn.Linear(256, latent_size)
        self.logvar_layer = nn.Linear(256, latent_size)

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_size + label_size, hidden_dim * 10000 * 88),  # Adjust the output size to match (batch_sz, 10000, 88)
            nn.ReLU(),
            nn.Unflatten(1, (10000, 88))  # Reshape to match the desired output shape
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x, label):
        # Encoding
        x = x.to(torch.float32)
        label = label.to(torch.float32)
        encoded = self.encoder(x)
        mu = self.mu_layer(encoded)
        logvar = self.logvar_layer(encoded)

        # Reparameterization trick
        z = self.reparameterize(mu, logvar)

        # Concatenate one-hot label with latent representation
        z_with_label = torch.cat((z, label), dim=1)

        # Decoding
        reconstructed = self.decoder(z_with_label)
        return reconstructed, mu, logvar




In [24]:
input_size = 10000 * 88  # Example input size
hidden_dim = 256
latent_size = 32
label_size = 4  # One-hot encoded label size
vae_model = VAE(input_size, hidden_dim, latent_size, label_size)

In [17]:
one_hot_labels = one_hot_encode(tag,4)
# print(one_hot_labels)
music = torch.tensor(music)
one_hot_labels = torch.tensor(one_hot_labels)

In [25]:
# Define your optimizer and loss function
optimizer = optim.Adam(vae_model.parameters(), lr=0.001)

# Reconstruction loss (MSE loss)
reconstruction_loss = nn.MSELoss()

# KL divergence loss
def kl_divergence_loss(mu, logvar):
    return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

# Create a DataLoader for batch training
dataset = TensorDataset(music, one_hot_labels)
dataloader = DataLoader(dataset, batch_size=3, shuffle=True)

# Number of training samples
total_samples = len(dataset)

# Training loop
epochs = 10
for epoch in range(epochs):
    vae_model.train()
    total_loss = 0.0
    for batch_music, batch_label in dataloader:
        optimizer.zero_grad()
        reconstructed, mu, logvar = vae_model(batch_music, batch_label)
        # Compute reconstruction loss
        recon_loss = reconstruction_loss(reconstructed, batch_music)
        # Compute KL divergence loss
        kl_loss = kl_divergence_loss(mu, logvar)
        # Total loss is the sum of reconstruction loss and KL divergence loss
        loss = recon_loss + kl_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(batch_music)
    average_loss = total_loss / total_samples
    print(f"Epoch {epoch + 1}/{epochs}, Total Loss: {average_loss:.4f}")

RuntimeError: unflatten: Provided sizes [10000, 88] don't multiply up to the size of dim 1 (225280000) in the input tensor